In [1]:
#ttps://eddwardo.github.io/posts/2019-06-05-finding-local-extreams-in-pandas-time-series/
import numpy as np
from scipy.signal import argrelextrema
from matplotlib import pyplot as plt
import pandas as pd
#%matplotlib inline
%matplotlib widget


In [2]:
min_period = 300
N = 14
df = pd.read_csv(f"data/USDT_BTC_{min_period}_1425168000_73.csv")
df.rename(columns={'date':'epoch','ts':'date'},inplace=True)
assert df.loc[0,'period'] == min_period, "Error, period different from min_period"
assert df['period'].count() == df.shape[0], "Error, period not always is min_period" 
df.drop(columns=['period','currency_pair','quoteVolume','weightedAverage'],inplace=True)
df['date'] = pd.to_datetime(df['date'])
df.sort_values(by="date") #just in case
df.set_index('date',inplace=True)
df

,epoch,high,low,open,close,volume
date,,,,,,
2015-03-01 00:00:00,1425168000,240.000060,240.000060,240.000060,240.000060,0.000000
2015-03-01 00:05:00,1425168300,240.000060,240.000060,240.000060,240.000060,0.000000
2015-03-01 00:10:00,1425168600,240.000060,240.000060,240.000060,240.000060,0.000000
2015-03-01 00:15:00,1425168900,240.000060,240.000060,240.000060,240.000060,0.000000
2015-03-01 00:20:00,1425169200,240.000060,240.000060,240.000060,240.000060,0.000000
...,...,...,...,...,...,...
2021-03-31 23:35:00,1617233700,58802.587059,58761.857003,58801.000000,58762.000000,29415.195318
2021-03-31 23:40:00,1617234000,58798.268464,58744.683295,58762.000000,58795.050899,169054.589184
2021-03-31 23:45:00,1617234300,58866.551166,58795.050899,58795.050899,58810.585631,4315.762275


In [3]:
daily_df = df.resample('D').agg({'epoch':'first','high':'max','low':'min','open':'first','close':'last','volume':'sum'})

In [4]:
def rsi(close,n):
    '''
    close is a pandas series
    n is the period for smooth in exponential means
    return a pandas series as rsi
    '''
    
    delta = close - close.shift(1) 
    delta = delta.dropna()
    
    closeup = delta.mask(delta < 0, 0)
    closedown = delta.mask(delta > 0,0).abs()
    

    closeup[closeup.index[n-1]]= closeup[:n].mean()
    closeup = closeup.drop(closeup.index[:n-1])
    avg_closeup = closeup.ewm(com=n-1,adjust=False).mean() 

    closedown[closedown.index[n-1]]= closedown[:n].mean()
    closedown = closedown.drop(closedown.index[:n-1])
    avg_closedown = closedown.ewm(com=n-1,adjust=False).mean()

    rsi = avg_closeup / (avg_closedown + avg_closeup)

    return rsi

In [5]:
 #+- 2 days -> weekly extrema for divergence better a smaller order for earlier detection
price_min_positions = argrelextrema(daily_df.low.values, np.less_equal, order=2)[0] 
price_max_positions = argrelextrema(daily_df.high.values, np.greater_equal, order=2)[0]

In [6]:
print(price_min_positions)
print(price_max_positions)

[   0   18   23   25   26   32   40   44   49   56   66   67   68   71
   74   79   85   93   97  102  103  111  115  125  129  132  140  144
  145  153  166  170  177  183  196  197  205  211  218  235  250  255
  259  260  264  269  276  286  295  300  305  308  311  321  327  334
  339  343  348  361  364  370  374  379  383  390  394  395  405  424
  428  438  447  464  471  480  483  486  491  494  509  513  520  524
  532  537  546  552  560  566  570  584  598  604  610  613  617  623
  630  636  645  651  670  678  682  696  711  715  722  726  740  748
  755  774  796  804  807  818  830  837  849  853  863  868  878  882
  898  902  905  911  919  924  929  936  949  962  969  972  987  999
 1004 1015 1027 1035 1044 1047 1053 1059 1062 1073 1092 1104 1110 1113
 1118 1127 1132 1143 1152 1157 1168 1174 1180 1185 1192 1200 1211 1215
 1229 1238 1244 1259 1262 1268 1278 1287 1298 1304 1312 1316 1321 1329
 1340 1343 1351 1355 1360 1365 1377 1385 1397 1402 1405 1414 1423 1430
 1440 

In [7]:
print(daily_df.iloc[price_min_positions].low)

date
2015-03-01      240.000060
2015-03-19      248.821580
2015-03-24      242.525500
2015-03-26      242.525500
2015-03-27      242.525500
                  ...     
2021-02-23    45000.000000
2021-02-28    43000.000000
2021-03-05    46325.000000
2021-03-16    53297.884124
2021-03-25    50483.383394
Name: low, Length: 298, dtype: float64


In [8]:
print(daily_df.iloc[price_max_positions].high)

date
2015-03-05      280.890000
2015-03-13      301.000000
2015-03-16      301.000000
2015-03-23      274.298098
2015-03-28      258.880679
                  ...     
2021-02-21    58320.000000
2021-03-03    52605.813647
2021-03-13    61799.000000
2021-03-18    60081.000000
2021-03-31    59782.490733
Name: high, Length: 318, dtype: float64


In [9]:
price_maximums = daily_df.iloc[price_max_positions].high
price_minimums = daily_df.iloc[price_min_positions].low

In [10]:
print(price_maximums)
print(type(price_maximums))

date
2015-03-05      280.890000
2015-03-13      301.000000
2015-03-16      301.000000
2015-03-23      274.298098
2015-03-28      258.880679
                  ...     
2021-02-21    58320.000000
2021-03-03    52605.813647
2021-03-13    61799.000000
2021-03-18    60081.000000
2021-03-31    59782.490733
Name: high, Length: 318, dtype: float64
<class 'pandas.core.series.Series'>


In [11]:
print(price_minimums)
print(type(price_minimums))

date
2015-03-01      240.000060
2015-03-19      248.821580
2015-03-24      242.525500
2015-03-26      242.525500
2015-03-27      242.525500
                  ...     
2021-02-23    45000.000000
2021-02-28    43000.000000
2021-03-05    46325.000000
2021-03-16    53297.884124
2021-03-25    50483.383394
Name: low, Length: 298, dtype: float64
<class 'pandas.core.series.Series'>


In [12]:
price_min_diff =  price_minimums - price_minimums.shift(1)
price_max_diff =  price_maximums - price_maximums.shift(1)

In [13]:
price_max_diff

date
2015-03-05             NaN
2015-03-13    2.011000e+01
2015-03-16   -9.000001e-08
2015-03-23   -2.670190e+01
2015-03-28   -1.541742e+01
                  ...     
2021-02-21    1.994000e+04
2021-03-03   -5.714186e+03
2021-03-13    9.193186e+03
2021-03-18   -1.718000e+03
2021-03-31   -2.985093e+02
Name: high, Length: 318, dtype: float64

In [14]:
daily_df['rsi'] = rsi(daily_df.close,14)

In [15]:
 #+- 2 days -> weekly extrema for divergence better a smaller order for earlier detection
rsi_min_positions = argrelextrema(daily_df.rsi.values, np.less_equal, order=2)[0] 
rsi_max_positions = argrelextrema(daily_df.rsi.values, np.greater_equal, order=2)[0]

In [16]:
rsi_maximums = daily_df.iloc[rsi_max_positions].rsi
rsi_minimums = daily_df.iloc[rsi_min_positions].rsi

In [17]:
rsi_min_diff =  rsi_minimums - rsi_minimums.shift(1)
rsi_max_diff =  rsi_maximums - rsi_maximums.shift(1)

In [18]:
rsi_max_diff

date
2015-03-22         NaN
2015-03-27   -0.065898
2015-03-30    0.010769
2015-04-03    0.008364
2015-04-06   -0.002002
                ...   
2021-03-03   -0.232204
2021-03-13    0.153025
2021-03-17   -0.096633
2021-03-20   -0.023660
2021-03-30   -0.002209
Name: rsi, Length: 341, dtype: float64

In [19]:
daily_df['correlationH'] = daily_df.high.rolling(3).corr(daily_df.rsi)
daily_df['correlationL'] = daily_df.low.rolling(3).corr(daily_df.rsi)

In [20]:
daily_df.dropna(inplace=True)
daily_df.correlationH.describe()
daily_df.correlationL.describe()

count    2207.000000
mean             inf
std              NaN
min        -0.999970
25%         0.293819
50%         0.785248
75%         0.965051
max              inf
Name: correlationL, dtype: float64

In [21]:

plt.style.use("seaborn")
fig, (ax1,ax2,ax3,ax4) = plt.subplots(nrows=4,ncols=1)
daily_df = daily_df.loc["2020-01":"2020-02"]
ax1.plot(daily_df.low)
ax2.plot(daily_df.high)
ax3.plot(daily_df.correlationH)
ax4.plot(daily_df.correlationL)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …